# Initialization

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider

provider = SaverProvider()

# Use the Simulator

In [5]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3

_RESETS = False

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.
Searching for ibm_sherbrooke and 23.10.27_07h46_300pts_2std


In [38]:
SHOTS = int(1e4)
LOGICAL = 1
P_AMBIG = 1
extreme_IQ_data = simulator.generate_extreme_IQ(SHOTS, P_AMBIG)
print(extreme_IQ_data.shape)

count_str: 000 00 00 00
(10000, 9)


In [39]:
from src import cpp_soft_info

cpp_soft_info.get_counts(extreme_IQ_data, simulator.qubit_mapping, simulator.grid_dict, simulator.processed_scaler_dict, ROUNDS)

{'111 11 11 11': 10000}

# qubit: point dictionary

In [48]:
import numpy as np

IQ_dict = {}
for qubit_idx, grid in grid_dict.items():
    # Calculate the differences
    grid_diff_max = grid.grid_density_0 - grid.grid_density_1
    grid_diff_min = grid.grid_density_1 - grid.grid_density_0

    # Apply coordinate restriction
    mask = (grid.grid_x > -1) & (grid.grid_x < 1) & (grid.grid_y > -1) & (grid.grid_y < 1)
    restricted_diff_max = np.where(mask, grid_diff_max, -np.inf)
    restricted_diff_min = np.where(mask, np.where(grid.grid_density_1 > grid.grid_density_0, grid_diff_min, np.inf), np.inf)

    # Find the maximum and minimum differences in the restricted grids
    max_diff = np.max(restricted_diff_max)
    min_diff = np.min(restricted_diff_min)

    # Get the coordinates of the maximum and minimum differences
    max_diff_coordinate = np.unravel_index(np.argmax(restricted_diff_max), grid_diff_max.shape)
    min_diff_coordinate = np.unravel_index(np.argmin(restricted_diff_min), grid_diff_min.shape)

    # Get the (x, y) coordinates for both
    x_max, y_max = grid.grid_x[max_diff_coordinate[0], max_diff_coordinate[1]], grid.grid_y[max_diff_coordinate[0], max_diff_coordinate[1]]
    x_min, y_min = grid.grid_x[min_diff_coordinate[0], min_diff_coordinate[1]], grid.grid_y[min_diff_coordinate[0], min_diff_coordinate[1]]

    # Get the densities at max and min coordinates
    density_0_safe = grid.grid_density_0[max_diff_coordinate[0], max_diff_coordinate[1]]
    density_1_safe = grid.grid_density_1[max_diff_coordinate[0], max_diff_coordinate[1]]
    density_0_at_min = grid.grid_density_0[min_diff_coordinate[0], min_diff_coordinate[1]]
    density_1_at_min = grid.grid_density_1[min_diff_coordinate[0], min_diff_coordinate[1]]

    # Get the scaler parameters
    (mean_real, std_real), (mean_imag, std_imag) = processed_scaler_dict[qubit_idx]

    # Inverse transform the coordinates
    x_max_rescaled = x_max * std_real + mean_real
    y_max_rescaled = y_max * std_imag + mean_imag
    x_min_rescaled = x_min * std_real + mean_real
    y_min_rescaled = y_min * std_imag + mean_imag

    # Create complex IQ points
    iq_point_max = complex(x_max_rescaled, y_max_rescaled)
    iq_point_min = complex(x_min_rescaled, y_min_rescaled)

    # Append or create new entry in result_dict
    if qubit_idx not in IQ_dict:
        IQ_dict[qubit_idx] = []

    IQ_dict[qubit_idx].append({
        "max_diff_0_coordinate": (x_max, y_max),
        "max_diff_0_coordinate_rescaled": (x_max_rescaled, y_max_rescaled),
        "min_diff_1_coordinate": (x_min, y_min),
        "min_diff_1_coordinate_rescaled": (x_min_rescaled, y_min_rescaled),
        "iq_point_safe": iq_point_max,
        "iq_point_ambig": iq_point_min,
        "density_0_safe": density_0_safe,
        "density_1_safe": density_1_safe,
        "density_0_ambig": density_0_at_min,
        "density_1_ambig": density_1_at_min
    })

# result_dict now contains the desired information for each qubit_idx within the restricted coordinate range
IQ_dict

{0: [{'max_diff_0_coordinate': (-0.528428093645485, 0.9966555183946486),
   'max_diff_0_coordinate_rescaled': (-21255695.339409884, -20904142.57871683),
   'min_diff_1_coordinate': (0.006688963210702337, -0.9297658862876255),
   'min_diff_1_coordinate_rescaled': (-3002981.1339821536,
    -41988324.393148735),
   'iq_point_safe': (-21255695.339409884-20904142.57871683j),
   'iq_point_ambig': (-3002981.1339821536-41988324.393148735j),
   'density_0_safe': -2.209176865838854,
   'density_1_safe': -9.33119916899053,
   'density_0_ambig': -5.334366172023276,
   'density_1_ambig': -5.333528319979654}],
 1: [{'max_diff_0_coordinate': (-0.9966555183946488, 0.20735785953177244),
   'max_diff_0_coordinate_rescaled': (-2371079.9779586853, -2118608.112790446),
   'min_diff_1_coordinate': (0.08695652173913038, -0.7290969899665551),
   'min_diff_1_coordinate_rescaled': (97323.31508834154, -3206955.230349723),
   'iq_point_safe': (-2371079.9779586853-2118608.112790446j),
   'iq_point_ambig': (97323.3

In [122]:
p_ambig = 0.1
_ambig = np.random.choice([0, 1], p=[1-p_ambig, p_ambig])
print(_ambig)

0


In [42]:
for qubit_idx, grid in grid_dict.items():
    # Existing calculations...

    # Get the scaler parameters
    (mean_real, std_real), (mean_imag, std_imag) = processed_scaler_dict[qubit_idx]

    # Inverse transform the coordinates
    x_max_rescaled = x_max * std_real + mean_real
    y_max_rescaled = y_max * std_imag + mean_imag
    x_min_rescaled = x_min * std_real + mean_real
    y_min_rescaled = y_min * std_imag + mean_imag

    # Create complex IQ points
    iq_point_max = complex(x_max_rescaled, y_max_rescaled)
    iq_point_min = complex(x_min_rescaled, y_min_rescaled)

    # Append or create new entry in result_dict
    if qubit_idx not in IQ_dict:
        IQ_dict[qubit_idx] = []

    IQ_dict[qubit_idx].append({
        "max_diff_0_coordinate": (x_max, y_max),
        "max_diff_0_coordinate_rescaled": (x_max_rescaled, y_max_rescaled),
        "min_diff_1_coordinate": (x_min, y_min),
        "min_diff_1_coordinate_rescaled": (x_min_rescaled, y_min_rescaled),
        "iq_point_max": iq_point_max,
        "iq_point_min": iq_point_min,
        "density_0_safe": density_0_safe,
        "density_1_safe": density_1_safe,
        "density_0_ambig": density_0_at_min,
        "density_1_ambig": density_1_at_min
    })

# result_dict now contains the appended results for each qubit_idx
IQ_dict


{0: [{'max_diff_0_coordinate': (0.6622073578595318, -0.8762541806020068),
   'max_diff_0_coordinate_rescaled': (19356593.767666813, -41402652.67608118),
   'min_diff_1_coordinate': (-0.04682274247491636, 0.5685618729096991),
   'min_diff_1_coordinate_rescaled': (-4828252.554524926, -25589516.315257248),
   'iq_point_max': (19356593.767666813-41402652.67608118j),
   'iq_point_min': (-4828252.554524926-25589516.315257248j),
   'density_0_safe': -2.575057260776477,
   'density_1_safe': -2.88603711078715,
   'density_0_ambig': -1.6551112836216095,
   'density_1_ambig': -1.655109187728499}],
 1: [{'max_diff_0_coordinate': (0.6622073578595318, -0.8762541806020068),
   'max_diff_0_coordinate_rescaled': (1407710.2484342942, -3377981.2059661807),
   'min_diff_1_coordinate': (-0.04682274247491636, 0.5685618729096991),
   'min_diff_1_coordinate_rescaled': (-207417.83220141468, -1698817.081731867),
   'iq_point_max': (1407710.2484342942-3377981.2059661807j),
   'iq_point_min': (-207417.83220141468

In [35]:
scaler = processed_scaler_dict[0]
scaler

((-3231140.06155, 34109759.67139307), (-31812278.3091, 10944740.212699896))

In [20]:
print(grid_dict.keys())

grid = grid_dict[0]
grid.grid_x.shape

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126])


(300, 300)

In [23]:
# argmax(p0-p1)

import numpy as np

grid_diff = grid.grid_density_0 - grid.grid_density_1

max_diff = np.max(grid_diff)

max_coordinate = np.unravel_index(np.argmax(grid_diff), grid_diff.shape)

print("max_coordinate:", max_coordinate)
print("max_diff:", max_diff)
print("max_diff_coordinate:", grid.grid_x[max_coordinate[0], max_coordinate[1]], grid.grid_y[max_coordinate[0], max_coordinate[1]])
print("density_0:", np.exp(grid.grid_density_0[max_coordinate[0], max_coordinate[1]]))
print("density_1:", np.exp(grid.grid_density_1[max_coordinate[0], max_coordinate[1]]))


print("max density_0:", np.max(grid.grid_density_0))
print("max density_1:", np.max(grid.grid_density_1))

max_coordinate: (214, 0)
max_diff: 10.630126629949016
max_diff_coordinate: -2.0 0.8628762541806019
density_0: 0.0020592818994954213
density_1: 4.978637028756986e-08
max density_0: -0.5714554436893611
max density_1: -0.9474639668506271


In [24]:
# argmax(p1-p0) such that p1 > p0

import numpy as np

# Assuming grid.grid_density_0 and grid.grid_density_1 are your p0 and p1 grids
p0 = grid.grid_density_0
p1 = grid.grid_density_1

# Step 1: Filter for points where p1 > p0
mask = p1 > p0

# Step 2: Compute the difference where p1 > p0
filtered_diff = np.where(mask, p1 - p0, np.inf)  # Use np.inf to ignore points where p1 <= p0

# Step 3: Find the minimum difference
min_diff = np.min(filtered_diff)

# Step 4: Get the coordinate of the minimum difference
min_coordinate = np.unravel_index(np.argmin(filtered_diff), filtered_diff.shape)

# Retrieving the (x, y) coordinates and the densities at the found point
x_coord = grid.grid_x[min_coordinate[0], min_coordinate[1]]
y_coord = grid.grid_y[min_coordinate[0], min_coordinate[1]]
density_0_at_min = p0[min_coordinate[0], min_coordinate[1]]
density_1_at_min = p1[min_coordinate[0], min_coordinate[1]]

print("Min coordinate:", min_coordinate)
print("Min diff:", min_diff)
print("Min diff coordinate (x, y):", x_coord, y_coord)
print("density_0 at min:", density_0_at_min)
print("density_1 at min:", density_1_at_min)


Min coordinate: (46, 150)
Min diff: 0.0003041507435073143
Min diff coordinate (x, y): 0.006688963210702337 -1.3846153846153846
density_0 at min: -5.834381338107666
density_1 at min: -5.834077187364159


# Generate IQ data

In [9]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
DISTANCE = 3
ROUNDS = 3
OTHER_DATE = '2021-11-11'
_RESETS = False

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, DISTANCE, ROUNDS, DEVICE, _is_hex=_is_hex, _resets = _RESETS, other_date=OTHER_DATE)

Found jobs for backend ibm_sherbrooke with closest execution date 2023-10-27 08:32:22.841567+00:00.


circuit = simulator.code.circuit["0"] 
circuit.draw(fold=-1)

In [ ]:
# Get noise model
p1Q = 0
pZ = 0
pRE = 0
pXY = 0

p2Q = 1e-2
# pRO = 15e-2
pRO = 0


# p1Q = 1e-4
# p2Q = 6e-3
# pXY = 1e-4
# pZ = 1e-4
# pRO = 1e-1
# pRE = 1e-3

noise_model = simulator.get_noise_model(p1Q, p2Q, pXY, pZ, pRO, pRE)

In [ ]:
SHOTS = int(1e4)
LOGICAL = 1
IQ_data = simulator.generate_IQ(SHOTS, noise_model=noise_model, logical=LOGICAL)

17:38:20 Warning: Getting counts via stim. This may take time.... IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/IQ_data/simulator.py, LINE: 46


100%|██████████| 20/20 [00:00<00:00, 484.13it/s]
